In [ ]:
import os
from pathlib import Path

import jax
import matplotlib.pyplot as plt
import numpy as np
import orbax.checkpoint as ocp
import pandas as pd
from jax import numpy as jnp

from blooms_ml.configs import classification_model
from blooms_ml.learning import create_train_state
from blooms_ml.utils import (
    labeling_binary_incremented,
)

### Get & Prepare data

In [ ]:
datadir = os.path.join(Path.home(), "blooms-ml_data")
df = pd.read_parquet(os.path.join(datadir, "data_ROHO/roho800_weekly_average_stacked.parquet"))
df = df.groupby(["station", "s_rho"]).apply(labeling_binary_incremented, include_groups=False)
df = df.reset_index().drop(columns="level_2")

In [ ]:
df_orig = pd.read_parquet(os.path.join(datadir, "data_ROHO/roho800_weekly_average.parquet"))

In [ ]:
df

In [ ]:
df_orig

In [ ]:
df = df[df['ocean_time'] > '2013-01-01']  # keep test data only
df_orig = df_orig[df_orig['ocean_time'] > '2013-01-01']  # keep test data only
df = df[df['label'].notna()]

In [ ]:
df['s_rho'].unique()

In [ ]:
df['station'].unique()

In [ ]:
station_number = 0

In [ ]:
df_station = df[df['station'] == station_number][df['s_rho'] == -0.02]
df_station.set_index('ocean_time', inplace=True)
df_station_orig = df_orig[df_orig['station'] == station_number][df_orig['s_rho'] == -0.02]
df_station_orig.set_index('ocean_time', inplace=True)

In [ ]:
df_station

In [ ]:
df_station_orig = df_station_orig[:-1]
df_station_orig

In [ ]:
df_obs = df_station.drop(columns=['station', 's_rho'])

In [ ]:
df_obs

In [ ]:
data = {
    'label': df_obs['label'].values,
    'observations': jnp.float32(df_obs.drop(columns=['label']).values),
}

### Inference

In [ ]:
checkpointdir = os.path.join(Path.home(), "blooms-ml_results/_bl5q3pg/chkpt_epoch_050")
orbax_checkpointer = ocp.StandardCheckpointer()
rng = jax.random.PRNGKey(0)  # jax.random.key(0)
rng, init_rng = jax.random.split(rng)

In [ ]:
state = create_train_state(init_rng, classification_model(), data['observations'].shape)
abstract_my_tree = jax.tree_util.tree_map(ocp.utils.to_shape_dtype_struct, state)

In [ ]:
state = orbax_checkpointer.restore(checkpointdir, args=ocp.args.StandardRestore(abstract_my_tree))

In [ ]:
logits = state.apply_fn({"params": state.params}, data['observations'])

### Visualization

In [ ]:
df_station["prediction"] = jax.nn.softmax(logits)[:, 1]
df_station_orig["prediction"] = np.where(df_station["prediction"] > 0.5, 1, 0)
df_station_orig["label"] = df_station["label"]
df_label = df_station_orig[df_station_orig["label"] == 1]
df_prediction = df_station_orig[df_station_orig["prediction"] == 1]

In [ ]:
_, ax1 = plt.subplots(figsize=(20, 5))
ax1.plot(df_station_orig.index, df_station_orig['P1_c'], 'b-')
ax1.plot(df_label.index, df_label['P1_c'], 'r^')
ax1.set_ylabel('Carbon', color='b')

ax2 = ax1.twinx()
ax2.plot(df_prediction.index, df_prediction['prediction'], 'cv')
ax2.set_ylabel('Prediction', color='c')

# savepath = os.path.join(Path.home(), f"tmp/blooms-ml_results/station_{station_number:04}.png")
# plt.savefig(savepath, dpi=300)